**looking around for stuff: Solr queries from Python**  

- Apache Solr Reference Guide > [Python](https://solr.apache.org/guide/solr/latest/deployment-guide/python.html)
    - this ^^^ references [`urllib2`](https://docs.python.org/2/library/urllib2.html), which is outdated...
    - """The urllib2 module has been split across several modules in Python 3 named urllib.request and urllib.error. 
    - ...see [`urllib.request`](https://docs.python.org/3/library/urllib.request.html), which in turn says...
    - """See also The [Requests package](https://requests.readthedocs.io/en/master/) is recommended for a higher-level HTTP client interface.


In [1]:
# a little very small test
import requests
od2solrprod = "https://solr-od2.library.oregonstate.edu/solr/prod/"

In [2]:
response = requests.get(f"{od2solrprod}select?q=identifier_tesim:PH395_UP_FB_20230902_*&rows=100")
print(type(response.json()))
print(response.json())


<class 'dict'>
{'responseHeader': {'zkConnected': True, 'status': 0, 'QTime': 2, 'params': {'q': 'identifier_tesim:PH395_UP_FB_20230902_*', 'rows': '100'}}, 'response': {'numFound': 64, 'start': 0, 'maxScore': 1.0, 'docs': [{'system_create_dtsi': '2025-03-07T20:14:58Z', 'system_modified_dtsi': '2025-03-21T00:57:09Z', 'has_model_ssim': ['Image'], 'id': '3b5919954', 'accessControl_ssim': ['b1e7be83-f149-4591-9262-66a2ce9b8e22'], 'hasRelatedMediaFragment_ssim': ['f03b5919954'], 'hasRelatedImage_ssim': ['f03b5919954'], 'depositor_ssim': ['briesenb@uoregon.edu'], 'depositor_tesim': ['briesenb@uoregon.edu'], 'title_tesim': ['Steve Stephens IV, 2023'], 'date_uploaded_dtsi': '2025-03-07T20:14:12Z', 'date_modified_dtsi': '2025-03-07T20:14:12Z', 'primarySet_ssim': ['uo'], 'hasEmbargo_ssim': ['afb7b27c-ac88-41d3-9087-3f3a4ce6380a'], 'hasLease_ssim': ['759c3208-0d65-48ec-a2fe-b9ba77a5f4f8'], 'photographer_tesim': ['http://opaquenamespace.org/ns/creator/HarveyPaulWIV'], 'photographer_label_tesim': 

In [3]:
# question - am I making the request anew each time I access var response??
print(response.json()['response']['numFound'])

64


**working on script to check imports** -- things to check:
- number of works in importer #
- any works with missing file sets?

In [ ]:
# another test
import requests, json
od2solrselect = "https://solr-od2.library.oregonstate.edu/solr/prod/select?"
q = "q=member_of_collection_ids_ssim:gb-warner-nosatsu&" # this could be arg set to '*:*' by default?
fq = ""
fl = "fl=title_tesim"
# using default JSON, not using wt
rows = "&rows=10"
response = requests.get(f"{od2solrselect}{q}{fq}{fl}{rows}")
pretty = json.dumps(response.json()['response']['docs'], indent=4)
print(pretty)
# if made into a function how to insert separators as needed along with q, fq, etc. arg values?
# or am I trying to reinvent the wheel

In [ ]:
# chunky workflow step one -- set importer number
importer = "3131"
# test func to print pretty
def prettyjson(solrresponse):
    to_print = json.dumps(solrresponse['response']['docs'], indent=4)
    print(to_print)

In [ ]:
# chunky workflow step two - check number of works and look for file_set_ids_ssim
response = requests.get(f"{od2solrselect}q=bulkrax_identifier_tesim:{importer}&rows=100")
# prettyjson(response.json()) # test
print(f"IMPORTER {importer}: {len(response.json()['response']['docs'])} works")
for doc in response.json()['response']['docs']:
    # print(type(doc)) # test
    if doc.get('file_set_ids_ssim') == None:
        print(f"PID {doc['id']}: no file set")


IMPORTER 3131: 53 works
PID jw827d21w: no file set


**getting all workType values**  
**getting all form_of_work values**  
*what fields do we need?*  

In [15]:
url = "https://solr-od2.library.oregonstate.edu/solr/prod/select?q=workType_tesim:*&fl=id,workType_tesim,workType_label_tesim,workType_parsable_label_ssim,institution_tesim&rows=10"
response = requests.get(url)
print(response.json())

{'responseHeader': {'zkConnected': True, 'status': 0, 'QTime': 80, 'params': {'q': 'workType_tesim:*', 'fl': 'id,workType_tesim,workType_label_tesim,workType_parsable_label_ssim,institution_tesim', 'rows': '10'}}, 'response': {'numFound': 370623, 'start': 0, 'docs': [{'id': 'df70pr70q', 'workType_tesim': ['http://vocab.getty.edu/aat/300028094'], 'workType_label_tesim': ['charts (maps)'], 'institution_tesim': ['http://id.loc.gov/authorities/names/n80017721'], 'workType_parsable_label_ssim': ['charts (maps)$http://vocab.getty.edu/aat/300028094']}, {'id': 'df70rr88h', 'workType_tesim': ['http://vocab.getty.edu/aat/300028094'], 'workType_label_tesim': ['charts (maps)'], 'institution_tesim': ['http://id.loc.gov/authorities/names/n80017721'], 'workType_parsable_label_ssim': ['charts (maps)$http://vocab.getty.edu/aat/300028094']}, {'id': 'df70rq67j', 'workType_tesim': ['http://vocab.getty.edu/aat/300028094'], 'workType_label_tesim': ['charts (maps)'], 'institution_tesim': ['http://id.loc.gov/

In [16]:
import requests, json
from datetime import date
od2solrselect = "https://solr-od2.library.oregonstate.edu/solr/prod/select?"
q = {
    "workType": "q=workType_tesim:*", 
    "form_of_work": "q=form_of_work_tesim:*" 
}
fl = {
    "workType": "&fl=id,workType_tesim,workType_label_tesim,workType_parsable_label_ssim,institution_tesim", 
    "form_of_work": "&fl=id,form_of_work_tesim,form_of_work_label_tesim,form_of_work_parsable_label_ssim,institution_tesim" 
}
# note for workType_tesim:*, "numFound":370624
today = date.today()
fd = today.strftime("%Y%m%d") # why funny color 
# print(fd) # test

**learning**  
- [datetime — Basic date and time types](https://docs.python.org/3/library/datetime.html)
    - [date Objects](https://docs.python.org/3/library/datetime.html#date-objects)
        - *classmethod* [date.today()](https://docs.python.org/3/library/datetime.html#datetime.date.today)
        - [date.strftime(format)](https://docs.python.org/3/library/datetime.html#datetime.date.strftime)

In [18]:
to_do = ["workType", "form_of_work"]
for query in to_do:
    response = requests.get(f"{od2solrselect}{q[query]}{fl[query]}&rows=371000")
    with open(f"../data_scratch/wt_fow/{fd}_{query}.json", "w+") as jsonfile:
        json.dump(response.json(), jsonfile)

In [22]:
with open("../data_scratch/wt_fow/20250520_workType.json", "r") as jsonfile:
    data = json.load(jsonfile)
print(type(data))
print(len(data['response']['docs']))

<class 'dict'>
370623
